# Numerical Methods

## Numerical Linear Algebra I

### Exercise solutions

In [1]:
import numpy as np
import scipy.linalg as sl


### <span style="color:blue">Exercise 1: Solving a linear system </span>
Formulate and solve the linear system and check you get the answer quoted above.

We need to form $A$ and $\boldsymbol{b}$ and perform the following

\begin{align}
A\boldsymbol{x} & = \boldsymbol{b}\\
\implies A^{-1}A\boldsymbol{x} & = A^{-1}\boldsymbol{b}\\
\implies I\boldsymbol{x} & = A^{-1}\boldsymbol{b}\\
\implies \boldsymbol{x} & = A^{-1}\boldsymbol{b}
\end{align}

so we can find the solution $\boldsymbol{x}$ by multiplying the inverse of $A$ with the RHS vector $\boldsymbol{b}$.

In [2]:
A=np.array([[2., 3.],[1., -4.]])

# check first whether the determinant of A is non-zero - see below for reasons why.
print(sl.det(A))   

-11.0


In [3]:
b=np.array([7., 3.])

# compute A inverse and multiply by b
print(sl.inv(A) @ b)

[3.36363636 0.09090909]


In [4]:
# or solve the linear system using scipy - actually does 
# the same thing as line above using LU decomposition (see later in lectures for the details)
print(sl.solve(A,b))

print("Check it against the solution we calculated by hand earlier: ", 37./11., 1./11.)

[3.36363636 0.09090909]
Check it against the solution we calculated by hand earlier:  3.3636363636363638 0.09090909090909091


In [5]:
# this is a more rigorous way of checking the result - using numpy.allclose:
print(np.allclose(np.array([37./11., 1./11.]), sl.solve(A,b)))

True


### <span style="color:blue">Exercise 2: Matrix manipulation in python </span>

Let
$$
A = \left(
  \begin{array}{ccc}
    1 & 2 & 3 \\
    4 & 5 & 6 \\
    7 & 8 & 9 \\
  \end{array}
\right)
\mathrm{\quad\quad and \quad\quad}
b = \left(
  \begin{array}{c}
    2 \\
    4 \\
    6 \\
  \end{array}
\right)
$$


- Store $A$ and $b$ in NumPy array structures. Print them.


- Print their shape and size. What is the difference ?


- Create a NumPy array $I$ containing the identity matrix $I_3$. Perform the update $A = A+I$.


- Substitute the 3rd column of $A$ with $b$.


- Solve the linear system $Ax=b$ - what's your solution?

In [6]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([2, 4, 6])
print("A =", A)
print("b = ",b)

print("Size of A: ", A.size," and shape of A: ",A.shape)
print("Size of b: ", b.size," and shape of b: ",b.shape)

I = np.eye(3)
print("I = ",I)
A = A + I
print("A = ",A)

A[:, 2] = b
print("A = ",A)

x = sl.solve(A,b)
print("x = ", x)

A = [[1 2 3]
 [4 5 6]
 [7 8 9]]
b =  [2 4 6]
Size of A:  9  and shape of A:  (3, 3)
Size of b:  3  and shape of b:  (3,)
I =  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
A =  [[ 2.  2.  3.]
 [ 4.  6.  6.]
 [ 7.  8. 10.]]
A =  [[2. 2. 2.]
 [4. 6. 4.]
 [7. 8. 6.]]
x =  [0. 0. 1.]


### <span style="color:blue">Exercise 3: Gaussian elimination $3 \times 3$ example (by hand) </span>

Consider the system of linear equations

\begin{align*}
  2x + 3y - 4z &= 5 \\
  6x + 8y + 2z &= 3 \\
  4x + 8y - 6z &= 19
\end{align*}

write this in matrix form, form the corresponding augmented system and perform row operations until you get to upper-triangular form, find the solution using back substitution (**do this all with pen and paper**).

Write some code to check your answer using `sl.inv(A) @ b`.

You should find $x=-6$, $y=5$, $z=-1/2$.

In [7]:
A = np.array([[2.,3.,-4.],[6.,8.,2.],[4.,8.,-6.]])
b = np.array([5.,3.,19.])
print(sl.inv(A) @ b)

[-6.   5.  -0.5]


### <span style="color:blue">Exercise 4: Gaussian elimination</span>

Write some code that takes a matrix $A$ and a vector $\boldsymbol{b}$ and converts it into upper-triangular form using the above algorithm. For the $2 \times 2$ and $3\times 3$ examples from above compare the resulting $A$ and $\boldsymbol{b}$ you obtain following elimination.


In [8]:
def upper_triangle(A, b):
    """ A function to covert A into upper triangluar form through row operations.
    The same row operations are performed on the vector b.
    
    Note that this implementation does not use partial pivoting which is introduced below.
    
    Also note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """
    n = np.size(b)
    rows, cols = np.shape(A)
    # check A is square
    assert(rows == cols)
    # and check A has the same numner of rows as the size of the vector b
    assert(rows == n)

    # Loop over each pivot row - all but the last row which we will never need to use as a pivot
    for k in range(n-1):
        # Loop over each row below the pivot row, including the last row which we do need to update
        for i in range(k+1, n):
            # Define the scaling factor for this row outside the innermost loop otherwise 
            # its value gets changed as you over-write A!!
            # There's also a performance saving from not recomputing things when not strictly necessary
            s = (A[i, k] / A[k, k])
            # Update the current row of A by looping over the column j
            # start the loop from k as we can assume the entries before this are already zero
            for j in range(k, n):
                A[i, j] = A[i, j] - s*A[k, j]
            # and update the corresponding entry of b
            b[i] = b[i] - s*b[k]


# Test our code on our 2x2 and 3x3 examples from above

# 2x2 example from above
A = np.array([[2., 3.], [1., -4.]])
b = np.array([7., 3.])

upper_triangle(A, b)


# Here is a new trick for you - "pretty print"
from pprint import pprint

print('Our A matrix following row operations to transform it into upper-triangular form:')
pprint(A)
print('The correspondingly updated b vector:')
pprint(b)

# 3x3 example from homework exercise
A = np.array([[2., 3., -4.],
              [6., 8., 2.],
              [4., 8., -6.]])
b = np.array([5., 3., 19.])


A = np.array( [ [2, 3, -4], [6, 8, 2], [4, 8, -6] ] )
b = np.array( [5, 3, 19] )

upper_triangle(A, b)

print('\nOur A matrix following row operations to transform it into upper-triangular form:')
pprint(A)
print('The correspondingly updated b vector:')
pprint(b)

Our A matrix following row operations to transform it into upper-triangular form:
array([[ 2. ,  3. ],
       [ 0. , -5.5]])
The correspondingly updated b vector:
array([ 7. , -0.5])

Our A matrix following row operations to transform it into upper-triangular form:
array([[ 2,  3, -4],
       [ 0, -1, 14],
       [ 0,  0, 30]])
The correspondingly updated b vector:
array([  5, -12, -15])


### <span style="color:blue">Exercise 5: Back substitution</span>

Extend your code to perform back substitution and hence to obtain the final solution $\boldsymbol{x}$.  Check against the solutions found earlier.  Come up with some random $n\times n$ matrices (you can use `np.random.rand` for that) and check your code against `sl.inv(A)@b` (remember to use the original $A$ and $\boldsymbol{b}$ here of course!)


In [9]:
# This function assumes that A is already an upper triangular matrix, 
# e.g. we have already run our upper_triangular function if needed.

def back_substitution(A, b):
    """ Function to perform back subsitution on the system Ax=b.
    
    Returns the solution x.
    
    Assumes that A is on upper triangular form.
    """
    n = np.size(b)
    # Check A is square and its number of rows and columns same as size of the vector b
    rows, cols = np.shape(A)
    assert(rows == cols)
    assert(rows == n)
    # We can/should check that A is upper triangular using np.triu which is the 
    # upper triangular part of a matrix - if A is already upper triangular, then
    # it should of course match the upper-triangular component of A!!
    assert(np.allclose(A, np.triu(A)))
    
    x = np.zeros(n)
    # start at the end (row n-1) and work backwards
    for k in range(n-1, -1, -1):
        # note that we could do this update in a single vectorised line 
        # using np.dot or @ - this could also speed things up
        s = 0.
        for j in range(k+1, n):
            s = s + A[k, j]*x[j]
        x[k] = (b[k] - s)/A[k, k]

    return x


# This A is the upper triangular matrix carried forward
# from the Python box above, and b the correspondingly updated b vector.
A = np.array([[ 2.,  3., -4.],
              [ 0., -1., 14.],
              [ 0.,  0., 30.]])
b = np.array([  5., -12., -15.])

# print the solution using our codes
x = back_substitution(A, b)
print('Our solution: ',x)  

# Reinitialise A and b !
# remember our functions overwrote them
A = np.array([[2., 3., -4.],
                 [6., 8., 2.],
                 [4., 8., -6.]])
b = np.array([5., 3., 19.])

# check our answer against what SciPy gives us by multiplying b by A inverse 
print('SciPy solution: ',sl.inv(A) @ b)

print('Success: ', np.allclose(x, sl.inv(A) @ b))

Our solution:  [-6.   5.  -0.5]
SciPy solution:  [-6.   5.  -0.5]
Success:  True


### <span style="color:blue">Exercise 6: Matrix inversion</span>

Try updating your code to construct the inverse matrix. Check your answer against the inverse matrix given by a built-in function.

Hint: Once you have performed your Gaussian elimination to transform A into an upper triangular matrix, perform another elimination "from bottom to top" to transform A into a diagonal matrix.

In [13]:
# This updated version of the upper_triangular function now
# assumes that a matrix, B, is in the old vector location (what was b)
# in the augmented system, and applies the same operations to
# B as to A - only a minor difference

def upper_triangle2(A, B):
    m, n = np.shape(A)
    assert(n == m)  # this is designed to work for a square matrix

    # Loop over each pivot row.
    for k in range(n-1):
        # Loop over each equation below the pivot row.
        for i in range(k+1, n):
            # Define the scaling factor outside the innermost
            # loop otherwise its value gets changed as you are
            # over-writing A
            s = (A[i, k]/A[k, k])
            for j in range(n):
                A[i, j] = A[i, j] - s*A[k, j]
                # replace the old b update with the same update as A
                B[i, j] = B[i, j] - s*B[k, j]


# and this is a version which transforms the matrix into lower
# triangular form - the point here is that if you give it a
# matrix that is already in upper triangular form, then the
# result will be a diagonal matrix
def lower_triangle2(A, B):
    m, n = np.shape(A)
    assert(n == m)  # this is designed to work for a square matrix

    # now it's basically just the upper triangular algorithm 
    # applied backwards
    for k in range(n-1, -1, -1):
        for i in range(k-1, -1, -1):
            s = (A[i, k]/A[k, k])
            for j in range(n):
                A[i, j] = A[i, j] - s*A[k, j]
                B[i, j] = B[i, j] - s*B[k, j]


# Let's redefine A as our matrix above
A = np.array([[2., 3., -4.], [3., -1., 2.], [4., 2., 2.]])

# and B is the identity of the corresponding size
B = np.eye(np.shape(A)[0])

# transform A into upper triangular form 
# (and perform the same operations on B)
upper_triangle2(A, B)
print('Upper triangular transformed A = ')
pprint(A)

# now make this updated A lower triangular as well 
# (the result should be diagonal)
lower_triangle2(A, B)
print('\nand following application of our lower triangular function = ')
pprint(A)

# The final step to achieve the identity is just to divide each row through by the value 
# of the diagonal to end up with 1's on the main diagonal and 0 everywhere else.
for i in range(np.shape(A)[0]):
    B[i, :] = B[i, :]/A[i, i]
    A[i, :] = A[i, :]/A[i, i]

# the final A should be the identity
print('\nOur final transformed A = ')
pprint(A)

# the final B should therefore be the inverse of the original B
print('\nand the correspondingly transformed B = ')
pprint(B)

# let's compute the inverse using built-in functions and check
# we get the same answer (we need to reinitialise A)
A = np.array([[2., 3., -4.], [3., -1., 2.], [4., 2., 2.]])
print('\nSciPy computes the inverse as:')
pprint(sl.inv(A))

# B should now store the inverse of the original A - let's check
print('\nSuccess: ', np.allclose(B, sl.inv(A)))

Upper triangular transformed A = 
array([[ 2.        ,  3.        , -4.        ],
       [ 0.        , -5.5       ,  8.        ],
       [ 0.        ,  0.        ,  4.18181818]])

and following application of our lower triangular function = 
array([[ 2.        ,  0.        ,  0.        ],
       [ 0.        , -5.5       ,  0.        ],
       [ 0.        ,  0.        ,  4.18181818]])

Our final transformed A = 
array([[ 1.,  0.,  0.],
       [-0.,  1., -0.],
       [ 0.,  0.,  1.]])

and the correspondingly transformed B = 
array([[ 0.13043478,  0.30434783, -0.04347826],
       [-0.04347826, -0.43478261,  0.34782609],
       [-0.2173913 , -0.17391304,  0.23913043]])

SciPy computes the inverse as:
array([[ 0.13043478,  0.30434783, -0.04347826],
       [-0.04347826, -0.43478261,  0.34782609],
       [-0.2173913 , -0.17391304,  0.23913043]])

Success:  True


### <span style="color:blue">Exercise 7: Zeros on the diagonal</span>

You may have noticed above that we have no way of guaranteeing that the $A_{kk}$ we divide through by in the Guassian elimination or back substitution algorithms is non-zero (or not very small which will also lead to computational problems).
Note also that we commented that we are free to exchange two rows in our augmented system - how could you use this fact to build robustness into our algorithms in order to deal with matrices for which our algorithms do lead to very small or zero $A_{kk}$ values?  

See if you can figure out how to do this - more on this next week!

In [11]:
# This function swaps rows in matrix A
# (and remember that we need to do likewise for the vector b 
# we are performing the same operations on)

def swap_row(A, b, i, j):
    """ Swap rows i and j of the matrix A and the vector b.
    """ 
    if i == j:
        return
    print('swapping rows', i,'and', j)
    # If we are swapping two values, we need to take a copy of one of them first otherwise
    # we will lose it when we make the first swap and will not be able to use it for the second.
    # We need to make sure it is a real copy - not just a copy of a reference to the data!
    # use np.copy to do this. 
    iA = np.copy(A[i, :])
    ib = np.copy(b[i])

    A[i, :] = A[j, :]
    b[i] = b[j]

    A[j, :] = iA
    b[j] = ib

    
# This is a new version of the upper_triangular function
# with the added step of swapping rows so the largest
# magnitude number is always our pivot/
# pp stands for partial pivoting which will be explained
# in more detail below.

def upper_triangle_pp(A, b):
    """ A function to covert A into upper triangluar form through row operations.
    The same row operations are performed on the vector b.
    
    This version uses partial pivoting.
    
    Note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """
    n = np.size(b)
    # check A is square and its number of rows and columns same as size of the vector b
    rows, cols = np.shape(A)
    assert(rows == cols)
    assert(rows == n)

    # Loop over each pivot row - all but the last row
    for k in range(n-1):
        # Swap rows so we are always dividing through by the largest number.
        # initiatise kmax with the current pivot row (k)
        kmax = k
        # loop over all entries below the pivot and select the k with the largest abs value
        for i in range(k+1, n):
            if abs(A[kmax, k]) < abs(A[i, k]):
                kmax = i
        # and swap the current pivot row (k) with the row with the largest abs value below the pivot
        swap_row(A, b, kmax, k)

        for i in range(k+1, n):
            s = (A[i, k]/A[k, k])
            for j in range(k, n):
                A[i, j] = A[i, j] - s*A[k, j]
            b[i] = b[i] - s*b[k]


# Apply the new code with row swaps to our matrix problem from above
A = np.array([[2., 3., -4.],
              [3., -1., 2.],
              [4., 2., 2.]])
b = np.array([10., 3., 8.])

upper_triangle_pp(A, b)

print('\nA and b with row swaps: ')
pprint(A)
pprint(b)
# compute the solution from these using our back substitution code
# could also have used SciPy of course
x1 = back_substitution(A, b)

# compare with our first function with no row swaps
A = np.array([[2., 3., -4.],
              [3., -1., 2.],
              [4., 2., 2.]])
b = np.array([10., 3., 8.])

upper_triangle(A, b)

print('\nA and b without any row swaps: ')
pprint(A)
pprint(b)
x2 = back_substitution(A, b)

# check these two systems are equivalent
print('\nThese two upper triangular systems are equivalent (i.e. have the same solution): ',np.allclose(x1, x2))

swapping rows 2 and 0

A and b with row swaps: 
array([[ 4. ,  2. ,  2. ],
       [ 0. , -2.5,  0.5],
       [ 0. ,  0. , -4.6]])
array([ 8. , -3. ,  3.6])

A and b without any row swaps: 
array([[ 2.        ,  3.        , -4.        ],
       [ 0.        , -5.5       ,  8.        ],
       [ 0.        ,  0.        ,  4.18181818]])
array([ 10.        , -12.        ,  -3.27272727])

These two upper triangular systems are equivalent (i.e. have the same solution):  True
